In [17]:
import numpy as np
num_frames = 156
left_bound = 0
right_bound = num_frames - 1
candidate_ids = np.arange(left_bound, right_bound)
nframe_view = 92

id_feat = np.random.choice(candidate_ids, size=1, replace=False)
print(id_feat)
candidate_ids = candidate_ids[(candidate_ids < (id_feat - nframe_view)) | (candidate_ids > (id_feat + nframe_view))]
print(candidate_ids.shape)

[10]
(52,)


In [11]:
uv = np.random.randn((256, 256, 2))

rix = np.random.permutation(uv.shape[0])
rix = rix[:1024]
uv = uv[rix]
rgb = rgb[rix]

(0,)